In [2]:
from selenium import webdriver
import time
import datetime
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup

In [3]:
browser = webdriver.Chrome('C:\\Users\\an-user\\Downloads\\chromedriver.exe')
browser.implicitly_wait(5)
browser.get('https://www.jobplanet.co.kr/')

In [28]:
title_list = []
advantage_list = []
drawback_list = []
opinion_list = []
personal_info = []
promotion, welfareandsalary, worklifebalance, corporateculture, executives = [], [],[],[],[]

In [29]:
for board_number in range(1, 2):
    browser.get("https://www.jobplanet.co.kr/companies/30139/reviews/%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90?page=&page=" + str(board_number))
    contents = browser.find_elements_by_css_selector('div.content_wrap')
    clean_contents = contents[:2] + contents[4:]

    for i in range(len(clean_contents)):
        ##personal_info and text info
        text = clean_contents[i].text.split('\n')
        personal_info.append(text[:text.index('평점')])
        title_list.append(''.join(str(text[12:text.index('장점')])))
        advantage_list.append(''.join(str(text[text.index('장점') + 1 : text.index('단점')])))
        drawback_list.append(''.join(str(text[text.index('단점') + 1 : text.index('경영진에 바라는 점')])))
        opinion_list.append(''.join(str(text[text.index('경영진에 바라는 점') + 1 : -5])))
    
        ##score info
        html = clean_contents[i].get_attribute('outerHTML')
        attributes = str(BeautifulSoup(html, 'lxml')).split('\n')
        promotion.append(re.findall(r'\d+', attributes[21]))
        welfareandsalary.append(re.findall(r'\d+', attributes[27]))
        worklifebalance.append(re.findall(r'\d+', attributes[33]))
        corporateculture.append(re.findall(r'\d+', attributes[39]))
        executives.append(re.findall(r'\d+', attributes[45]))



In [30]:
personal_info_df = pd.DataFrame(personal_info, columns = ['info'])
title_df = pd.DataFrame(title_list, columns = ['title'])
advantage_df = pd.DataFrame(advantage_list, columns = ['advantage'])
drawback_df = pd.DataFrame(drawback_list, columns = ['drawback'])
opinion_df = pd.DataFrame(opinion_list, columns = ['opinion'])
promotion_score = pd.DataFrame(promotion, columns = ['promotion'])
welfareandsalary_score = pd.DataFrame(welfareandsalary, columns = ['welfare salar'])
worklifebalance_score = pd.DataFrame(worklifebalance, columns = ['worklifebalance'])
corporateculture_score = pd.DataFrame(corporateculture, columns = ['culture'])
executives_score = pd.DataFrame(executives, columns = ['executives'])

In [31]:
high_df1 = pd.concat([personal_info_df,
                     title_df,
                     advantage_df,
                     drawback_df,
                     opinion_df,
                     promotion_score,
                     welfareandsalary_score,
                     worklifebalance_score,
                     corporateculture_score,
                     executives_score], axis=1)


In [32]:
high_df1.head()

,info,title,advantage,drawback,opinion,promotion,welfare salar,worklifebalance,culture,executives
0,인사/총무 | 현직원 | 경기 | 2019/8/23,"['""국내에서 손꼽히는 기업인 만큼 다양한 인재가 모여있는 곳이다.""']",['외국계 회사의 복지는 알지 못하지만 뒤쳐지지 않는 복지를 가지고 있다고 생각한다.'],"['행정서무 인력을 비효율적으로 2년마다 변경하는 방식이며, 몇가지 업무 처리 프로...",['행정서무 인력이 조금 더 실력을 인정받으면 다른 대우를 받았으면 합니다.'],40,100,100,80,100
1,IT/인터넷 | 현직원 | 경기 | 2019/8/21,"['""복지가 좋고 연봉이 높습니다. 기업분위기도 좋습니다""']","['식대 하루 세번 무료에 복지가 좋습니다', '성과금이 많이 나와서 연봉도 높습니다']","['단순 보고를 위한 업무가 많아 비효율적입니다', '제조회사 프로세스에 최적화되어...",['신규 사업 의지에 적극적인 투자를 지원해줬으면 합니다'],80,100,80,60,40
2,영업/제휴 | 전직원 | 경기 | 2019/8/21,"['""시스템으로 모든 것이 돌아가는 회사. 시스템은 국내 기업 중에 가장 탄탄한 거...",['삼성이라는 브랜드가 국내외 이직하기에 도움이 많이 됩니다.'],['굉장히 딱딱한 조직문화. 상사가 퇴근하기 전에는 집에 갈 생각 못함'],"['조직문화 개선', '부조리한 업무 형태 개선']",60,80,20,40,60
3,엔지니어링 | 현직원 | 경기 | 2019/8/21,"['""돈만큼 일해야합니다. 하지만 급여와 복지는 확실합니다.""']","['돈은 정말 만족스럽게 주는 게 큰 장점입니다.', '일이 힘들어도 월급날 보면서...","['그치만 그만큼 일이 힘든건 감안 하셔야 합니다.', '부서마다 아직 수직적 구조...",['신규 사업 추진에 있어서 조금 더 빠르게 소식을 전해줬으면 좋겠습니다.'],100,100,80,80,80
4,영업/제휴 | 전직원 | 경기 | 2019/8/21,"['""나오면 알게 될 것이다, 갓성전자라는 말이 괜히 있는게 아니다.""']","['역시 유연근무제. 자율출퇴근.', '다양한 아이디어 제출과 빠른 수렴과 수정과 ...","['어딜가나 있는 사내 정치질과 줄서기.', '근무시간이 부족할 정도의 초과근무.']","['미래 지향적인 아이템에 대해 과감한 투자를 해야할 것 같음.', '안되겠다 싶으...",60,100,80,100,40
